## Especialización en Sistemas de Información Geográfica  
Laboratorio SIG  
Taller 03 - IMPACTO PARQUE NACIONAL POR EL USO DE LEÑA  
RAFAEL FABIAN SANCHEZ OSORIO

### 0. INTRODUCCIÓN

En la zona se desarrollan actividades agrícolas intensas de las cuales se obtiene leña para cocinar, 
desafortunadamente la leña obtenida de la agricultura no es suficiente para satisfacer la demanda de 
los habitantes de la zona. La única fuente de leña es el parque nacional donde no está permitida la 
recolección de leña (tala de vegetación leñosa), sin embargo, la gente ingresa ilegalmente al parque 
para abastecerse de leña.  
Para poder definir un manejo que reduzca el impacto en el parque nacional, se tiene que determinar las 
áreas principales con déficit de rendimiento de leña. Últimamente se está notando un cambio en la 
agricultura que agrava los problemas. Muchas de las zonas con plantaciones de café y cítricos (de alto 
rendimiento de leña) se están cambiando a cultivos como los frutales y hierbas aromáticas, con lo cual 
se logra una mayor ganancia económica, pero un menor rendimiento de leña, causando una presión 
más alta al parque nacional.

### 1 Insumos Modelo

In [1]:
import arcpy
from arcgis.gis import GIS
import os
import pandas as pd

gis = GIS()
arcpy.env.overwriteOutput = True

Insumos

In [2]:
geodatabaseInsumos = r'C:\Users\rfabi\Downloads\DATOS_T3.gdb'
veredas = os.path.join(geodatabaseInsumos,'VEREDAS_2021')
usos = os.path.join(geodatabaseInsumos,'USOS_2021')
poblados = os.path.join(geodatabaseInsumos,'POBLADOS_2021')
DTM10 = os.path.join(geodatabaseInsumos,'DEM')
vias = os.path.join(geodatabaseInsumos,'VIAS_2021')
tablaPoblacion = os.path.join(geodatabaseInsumos,'TAB_POBLACION')
tablaRendimiento = os.path.join(geodatabaseInsumos,'TAB_RENDIMIENTO')
tablaVelocidadVias = os.path.join(geodatabaseInsumos,'TAB_VELOCIDAD_VIAS')
tablaVelocidadUsos = os.path.join(geodatabaseInsumos,'TAB_VELOCIDAD_USOS')

Salidas

In [3]:

GDBResultados = r'C:\Users\rfabi\OneDrive\Documentos\Universidad\00 Esp SIG\00 LabSIG\03Taller\Resultados.gdb'
usosVeredas = os.path.join(GDBResultados,'UsosVeredas')
usos = os.path.join(geodatabaseInsumos,'USOS_2021')
poblados = os.path.join(geodatabaseInsumos,'POBLADOS_2021')
DTM10 = os.path.join(geodatabaseInsumos,'DEM')
vias = os.path.join(geodatabaseInsumos,'VIAS_2021')
tablaPoblacion = os.path.join(geodatabaseInsumos,'TAB_POBLACION')
tablaRendimiento = os.path.join(geodatabaseInsumos,'TAB_RENDIMIENTO')
tablaVelocidadVias = os.path.join(geodatabaseInsumos,'TAB_VELOCIDAD_VIAS')
tablaVelocidadUsos = os.path.join(geodatabaseInsumos,'TAB_VELOCIDAD_USOS')


veredasDemandaLena = os.path.join(GDBResultados,'DemandaLena')
pobladosDemanda = os.path.join(GDBResultados,'PobladosDemanda')
pobladosDeficit = os.path.join(GDBResultados,'PobladosDeficit')
parqueNacional = os.path.join(GDBResultados,'ParqueNacional')
parqueNacionalTiempos = os.path.join(GDBResultados,'ParqueNacionalTiempos')
accesibilidadPNN = os.path.join(GDBResultados,'AccesibilidadPNN')
accesibilidadPNNRaster = os.path.join(GDBResultados,'AccesibilidadPNNRaster')
veredasDemanda2025 = os.path.join(GDBResultados,'VeredasDemanda2025')
usosCambio = os.path.join(GDBResultados,'UsosCambio')
pobladosParqueNacional = os.path.join(GDBResultados,'PobladosPNN')


pendienteRaster = os.path.join(GDBResultados,'PendienteRaster')
PendienteReclassRaster = os.path.join(GDBResultados, 'PendientesReclasficadas')
PendientesCorregidas = os.path.join(GDBResultados,'PendienteCorregidas')
ViasRaster = os.path.join(GDBResultados,'ViasRaster')
UsosRaster = os.path.join(GDBResultados,'UsosRaster')
VelocidadesRaster = os.path.join(GDBResultados,'Velocidades')
VelocidadesAjustadasRaster = os.path.join(GDBResultados,'VelocidadesAjustadas')
PesosRaster = os.path.join(GDBResultados,'Pesos')
TiemposData = os.path.join(GDBResultados,'Tiempos')

### 2.1 Disponibilidad de Leña
Para obtener la disponibilidad de leña se necesita la capa de uso y la información sobre el rendimiento 
de leña por cada tipo de uso por hectárea por año. También se tienen que utilizar el dato de consumo 
per cápita y la información de población disponible a nivel de unidad administrativa. Este análisis se 
realiza a nivel de unidad administrativa.



In [4]:
arcpy.analysis.TabulateIntersection(veredas, 'VEREDA', usos, usosVeredas, 'CLASES_USO')
veredasUSOData = arcpy.da.TableToNumPyArray(usosVeredas,('VEREDA','CLASES_USO','AREA'))
tablaRendimientoData = arcpy.da.TableToNumPyArray(tablaRendimiento,('CLASES_USO','RENDI'))
tablaPoblacionData = arcpy.da.TableToNumPyArray(tablaPoblacion,('VEREDA','POP_05','POP_01','POP_10','POB_2021'))

DFVeredadUSO = pd.DataFrame(veredasUSOData).infer_objects()
DFRendimientos = pd.DataFrame(tablaRendimientoData).infer_objects()
DFPoblacion = pd.DataFrame(tablaPoblacionData).infer_objects()

DFVeredadUSO['AREA'] = DFVeredadUSO['AREA'].apply(lambda x: round(x/10000,5))

Disponibilidad de Leña

In [5]:

DFVeredasRendimiento = pd.merge(DFVeredadUSO, DFRendimientos, on = 'CLASES_USO', how='left')
DFVeredasRendimiento['Rendimiento'] = DFVeredasRendimiento['AREA']*DFVeredasRendimiento['RENDI']
DFVeredasRendimiento = DFVeredasRendimiento.groupby('VEREDA').sum().reset_index()
DFVeredasRendimiento = DFVeredasRendimiento.drop('RENDI', axis=1)
DFVeredasRendimiento.head(10)

,VEREDA,AREA,Rendimiento
0,EL CAJON,7568.59245,991.861174
1,EL HATO,7101.11925,8885.496879
2,HATO VIEJO,2541.53438,486.704160
3,OJO DE AGUA,5609.81089,1078.546126
4,PALMERAS,9534.18300,9974.767633
5,PARAMO BAJO,10302.51710,2233.589117
6,PNN,1805.54568,0.000000
7,RASGATA ALTO,8938.87785,10920.872655
8,SALINAS,8569.16140,1770.915830


In [6]:
mapaRendimientoLeña = gis.map(location='Chiscas Boyacá')
arcpy.management.AddField(veredas, 'Rendimiento', 'FLOAT')
with arcpy.da.UpdateCursor(veredas, ['VEREDA','Rendimiento']) as cursor:
    for row in cursor:
        row[1] = DFVeredasRendimiento.loc[DFVeredasRendimiento['VEREDA'] == row[0]]['Rendimiento'].values[0]
        cursor.updateRow(row)
datosRendimiento = pd.DataFrame.spatial.from_featureclass(veredas)
datosRendimiento.spatial.plot(map_widget = mapaRendimientoLeña, col='Rendimiento', class_count=10,renderer_type='c',line_width=0.2)
mapaRendimientoLeña.legend = True
mapaRendimientoLeña

MapView(layout=Layout(height='400px', width='100%'), legend=True)

### 2.2 Demanda de Leña

In [7]:
DFVeredasRendimiento = pd.merge(DFVeredasRendimiento, DFPoblacion, on = 'VEREDA', how='left')
DFVeredasRendimiento['Consumo'] = DFVeredasRendimiento['POB_2021']*0.5*0.97
DFVeredasRendimiento['Demanda'] = DFVeredasRendimiento['Consumo']-DFVeredasRendimiento['Rendimiento']

arcpy.management.AddField(veredas, 'Consumo', 'FLOAT')
arcpy.management.AddField(veredas, 'Demanda', 'FLOAT')
with arcpy.da.UpdateCursor(veredas, ['VEREDA','Consumo','Demanda']) as cursor:
    for row in cursor:
        row[1] = DFVeredasRendimiento.loc[DFVeredasRendimiento['VEREDA'] == row[0]]['Consumo'].values[0]
        row[2] = DFVeredasRendimiento.loc[DFVeredasRendimiento['VEREDA'] == row[0]]['Demanda'].values[0]
        cursor.updateRow(row)
mapaDemandaLeña = gis.map(location='Chiscas Boyacá')
datosRendimiento = pd.DataFrame.spatial.from_featureclass(veredas)
DFVeredasRendimiento.head(10)

,VEREDA,AREA,Rendimiento,POP_05,POP_01,POP_10,POB_2021,Consumo,Demanda
0,EL CAJON,7568.59245,991.861174,553,162,1493,6563,3183.055,2191.193826
1,EL HATO,7101.11925,8885.496879,2127,431,2877,10220,4956.700,-3928.796879
2,HATO VIEJO,2541.53438,486.704160,681,389,1222,2621,1271.185,784.480840
3,OJO DE AGUA,5609.81089,1078.546126,696,402,1333,2964,1437.540,358.993874
4,PALMERAS,9534.18300,9974.767633,784,285,1553,4809,2332.365,-7642.402633
5,PARAMO BAJO,10302.51710,2233.589117,973,678,1617,6124,2970.140,736.550883
6,PNN,1805.54568,0.000000,0,0,0,0,0.000,0.000000
7,RASGATA ALTO,8938.87785,10920.872655,610,520,1049,2677,1298.345,-9622.527656
8,SALINAS,8569.16140,1770.915830,635,346,1273,4300,2085.500,314.584170


In [8]:
arcpy.analysis.Select(veredas, veredasDemandaLena, 'Demanda > 0')
datosDemanda = pd.DataFrame.spatial.from_featureclass(veredasDemandaLena)
datosDemanda.spatial.plot(map_widget = mapaDemandaLeña, col='Demanda', class_count=10,renderer_type='c',line_width=0.2)
mapaDemandaLeña.legend = True
mapaDemandaLeña

MapView(layout=Layout(height='400px', width='100%'), legend=True)

### 2.3 Accesibilidad a la Leña

2.3.1 Generando raster con los valores de corrección en velocidad por pendiente

In [9]:
pendiente = arcpy.sa.Slope(DTM10,"DEGREE")
pendiente.save(pendienteRaster)

pendienteReclasificadas = arcpy.sa.Reclassify(pendienteRaster, "Value",
    arcpy.sa.RemapRange(
        [[0,5,1],
        [5,10,2],
        [10,20,3],
        [20,30,4],
        [30,45,5],
        [45,65,6],
        [65,800,7]]))
pendienteReclasificadas.save(PendienteReclassRaster)

arcpy.management.AddField(PendienteReclassRaster, 'Correccion','DOUBLE')

correccionPendiente = {
    1 : 1,
    2 : 0.958,
    3 : 0.818,
    4 : 0.646,
    5 : 0.5,
    6 : 0.409,
    7 : 0.288
}

with arcpy.da.UpdateCursor(PendienteReclassRaster, ['Correccion', 'Value']) as cursor:
    for row in cursor:
        row[0] = correccionPendiente[row[1]]
        cursor.updateRow(row)

CorreccionPendienteData = arcpy.sa.Lookup(PendienteReclassRaster,'Correccion')
CorreccionPendienteData.save(PendientesCorregidas)

2.3.2 Generando Raster con las velocidades asignadas por clase de uso y vías

In [10]:
arcpy.conversion.FeatureToRaster(vias, 'VELOCIDAD', ViasRaster,12.5)
arcpy.conversion.FeatureToRaster(usos, 'VELOCIDAD', UsosRaster,12.5)
Velocidades = arcpy.sa.RasterCalculator([ViasRaster, UsosRaster],['x','y'],'Con(IsNull(x) ,y, x)')
Velocidades.save(VelocidadesRaster)
velocidadesAjustadas = arcpy.sa.Times(PendientesCorregidas,Velocidades)
velocidadesAjustadas.save(VelocidadesAjustadasRaster)
Pesos = arcpy.sa.RasterCalculator([VelocidadesAjustadasRaster],['x'],'Con(x==0,0.00001,3.6/x)')
Pesos.save(PesosRaster)

2.3.3 Extrayendo poblados que se encuentran dentro de las veredas con demanda

In [11]:
arcpy.analysis.Select(poblados, pobladosDemanda, "POBLADO NOT LIKE '%X'")
arcpy.analysis.Clip(pobladosDemanda, veredasDemandaLena, pobladosDeficit)
tiempos = arcpy.sa.CostDistance(pobladosDeficit,Pesos)
tiempos.save(TiemposData)

2.3.4 Extrayendo Parque Nacional

In [12]:
arcpy.analysis.Select(usos, parqueNacional, "CLASES_USO = 'PNN'")
parqueNacionalTime = arcpy.sa.ExtractByMask(TiemposData, parqueNacional)
parqueNacionalTime.save(parqueNacionalTiempos)
accesibilidadPNNRasterData = arcpy.sa.RasterCalculator([parqueNacionalTime],['x'],'Int(x)')
accesibilidadPNNRasterData.save(accesibilidadPNNRaster)

2.3.5 Estimando tiempos de Accesibilidad al Parque Nacional

In [13]:

arcpy.management.AddField(accesibilidadPNNRaster, 'AreaHa', 'DOUBLE')
arcpy.management.AddField(accesibilidadPNNRaster, 'AreaAcumulada', 'DOUBLE')
arcpy.management.AddField(accesibilidadPNNRaster, 'Impacto', 'TEXT')

def ImpactoArea(Area):
    if Area < 100:
        return '00 IMPACTO ALTO'
    elif (Area > 100 and Area<200):
        return '01 IMPACTO MEDIO'
    elif (Area>200 and Area<300):
        return '02 IMPACTO BAJO'
    else:
        return '03 SIN IMPACTO'  

with arcpy.da.UpdateCursor(accesibilidadPNNRaster, ['Count', 'AreaHa','AreaAcumulada']) as cursor:
    for row in cursor:
        area = 0
        row[1] = row[0]*12.5*12.5/10000
        area = area + row[1]
        row[2] = area
        cursor.updateRow(row)

with arcpy.da.UpdateCursor(accesibilidadPNNRaster, ['AreaHa','AreaAcumulada']) as cursor:
    area = 0
    for row in cursor:
        area = area+row[0]
        row[1] = area
        cursor.updateRow(row)
with arcpy.da.UpdateCursor(accesibilidadPNNRaster, ['AreaAcumulada','Impacto']) as cursor:
    for row in cursor:
        row[1] = ImpactoArea(row[0])
        cursor.updateRow(row)

arcpy.conversion.RasterToPolygon(accesibilidadPNNRaster,accesibilidadPNN,'','Impacto')

<Result 'C:\\Users\\rfabi\\OneDrive\\Documentos\\Universidad\\00 Esp SIG\\00 LabSIG\\03Taller\\Resultados.gdb\\AccesibilidadPNN'>

In [14]:

mapaAccesibilidadPNN = gis.map(location='Chiscas Boyacá')
zonasImpactoPNN = pd.DataFrame.spatial.from_featureclass(accesibilidadPNN)
zonasImpactoPNN.spatial.plot(map_widget = mapaAccesibilidadPNN, col='Impacto', class_count=10,renderer_type='u',line_width=0.2)
mapaAccesibilidadPNN.legend = True
mapaAccesibilidadPNN

MapView(layout=Layout(height='400px', width='100%'), legend=True)

### 3.0 Alternativa 02 (Uso de estufas eficientes)

Se plantea utilizar la estufa de leña eficiente huellas que presenta un ahorro de leña del 43.4%

3.1 Proyección Crecimiento Poblacional

Se utiliza el metodo de proyección recomendado por la RAS para la proyección de poblaciones pequeñas con poco crecimiento (Metodo Aritmetico) y se elimina de la Vereda Salinas

In [15]:
DFVeredasDemanda2025 = DFVeredasRendimiento
DFVeredasDemanda2025['POB_2025'] = DFVeredasDemanda2025['POB_2021']+((DFVeredasDemanda2025['POB_2021']-DFVeredasDemanda2025['POP_01'])/(2021-2001))*(2025-2021)
DFVeredasDemanda2025['POB_2025'] = DFVeredasDemanda2025['POB_2025'].astype(int)
DFVeredasRendimiento = DFVeredasDemanda2025
DFVeredasRendimiento['Consumo2025'] =  DFVeredasRendimiento['POB_2025']*0.5*0.97
DFVeredasRendimiento['Demanda2025'] =  DFVeredasRendimiento['Consumo2025']-DFVeredasRendimiento['Rendimiento']
DFVeredasDemanda2025 = DFVeredasDemanda2025.drop(8)
DFVeredasDemanda2025.head(10)

,VEREDA,AREA,Rendimiento,POP_05,POP_01,POP_10,POB_2021,Consumo,Demanda,POB_2025,Consumo2025,Demanda2025
0,EL CAJON,7568.59245,991.861174,553,162,1493,6563,3183.055,2191.193826,7843,3803.855,2811.993826
1,EL HATO,7101.11925,8885.496879,2127,431,2877,10220,4956.700,-3928.796879,12177,5905.845,-2979.651878
2,HATO VIEJO,2541.53438,486.704160,681,389,1222,2621,1271.185,784.480840,3067,1487.495,1000.790840
3,OJO DE AGUA,5609.81089,1078.546126,696,402,1333,2964,1437.540,358.993874,3476,1685.860,607.313874
4,PALMERAS,9534.18300,9974.767633,784,285,1553,4809,2332.365,-7642.402633,5713,2770.805,-7203.962633
5,PARAMO BAJO,10302.51710,2233.589117,973,678,1617,6124,2970.140,736.550883,7213,3498.305,1264.715883
6,PNN,1805.54568,0.000000,0,0,0,0,0.000,0.000000,0,0.000,0.000000
7,RASGATA ALTO,8938.87785,10920.872655,610,520,1049,2677,1298.345,-9622.527656,3108,1507.380,-9413.492656


Con la problación de la población al 2025, se procede a estimar el nuevo consumo de leña

In [16]:
DFVeredasDemanda2025['Consumo2025'] =  DFVeredasDemanda2025['POB_2025']*0.5*0.97
DFVeredasDemanda2025.head(10)

,VEREDA,AREA,Rendimiento,POP_05,POP_01,POP_10,POB_2021,Consumo,Demanda,POB_2025,Consumo2025,Demanda2025
0,EL CAJON,7568.59245,991.861174,553,162,1493,6563,3183.055,2191.193826,7843,3803.855,2811.993826
1,EL HATO,7101.11925,8885.496879,2127,431,2877,10220,4956.700,-3928.796879,12177,5905.845,-2979.651878
2,HATO VIEJO,2541.53438,486.704160,681,389,1222,2621,1271.185,784.480840,3067,1487.495,1000.790840
3,OJO DE AGUA,5609.81089,1078.546126,696,402,1333,2964,1437.540,358.993874,3476,1685.860,607.313874
4,PALMERAS,9534.18300,9974.767633,784,285,1553,4809,2332.365,-7642.402633,5713,2770.805,-7203.962633
5,PARAMO BAJO,10302.51710,2233.589117,973,678,1617,6124,2970.140,736.550883,7213,3498.305,1264.715883
6,PNN,1805.54568,0.000000,0,0,0,0,0.000,0.000000,0,0.000,0.000000
7,RASGATA ALTO,8938.87785,10920.872655,610,520,1049,2677,1298.345,-9622.527656,3108,1507.380,-9413.492656


Obtenido el consumo con la proyección al 2025, se procede a estimar el consumo utilizando la estufa de leña eficiente Huellas que presenta un ahorro del 43.4%, se asume el mismo rendimiento de las clases de uso actual


In [17]:
DFVeredasDemanda2025['Demanda2025'] =  DFVeredasDemanda2025['Consumo2025']-DFVeredasDemanda2025['Rendimiento']
DFVeredasDemanda2025['Demanda2025EstufaEficiente'] =  DFVeredasDemanda2025['Consumo2025']*0.434-DFVeredasDemanda2025['Rendimiento']
DFVeredasDemanda2025.head(10)

,VEREDA,AREA,Rendimiento,POP_05,POP_01,POP_10,POB_2021,Consumo,Demanda,POB_2025,Consumo2025,Demanda2025,Demanda2025EstufaEficiente
0,EL CAJON,7568.59245,991.861174,553,162,1493,6563,3183.055,2191.193826,7843,3803.855,2811.993826,659.011896
1,EL HATO,7101.11925,8885.496879,2127,431,2877,10220,4956.700,-3928.796879,12177,5905.845,-2979.651878,-6322.360148
2,HATO VIEJO,2541.53438,486.704160,681,389,1222,2621,1271.185,784.480840,3067,1487.495,1000.790840,158.868670
3,OJO DE AGUA,5609.81089,1078.546126,696,402,1333,2964,1437.540,358.993874,3476,1685.860,607.313874,-346.882886
4,PALMERAS,9534.18300,9974.767633,784,285,1553,4809,2332.365,-7642.402633,5713,2770.805,-7203.962633,-8772.238263
5,PARAMO BAJO,10302.51710,2233.589117,973,678,1617,6124,2970.140,736.550883,7213,3498.305,1264.715883,-715.324747
6,PNN,1805.54568,0.000000,0,0,0,0,0.000,0.000000,0,0.000,0.000000,0.000000
7,RASGATA ALTO,8938.87785,10920.872655,610,520,1049,2677,1298.345,-9622.527656,3108,1507.380,-9413.492656,-10266.669735


In [18]:
graficoConsumoLeñaSinAhorro = DFVeredasDemanda2025.plot.bar(x='VEREDA',y='Demanda2025', rot = 0, figsize=(12,8))

In [19]:
graficoConsumoLeñaConAhorro = DFVeredasDemanda2025.plot.bar(x='VEREDA',y='Demanda2025EstufaEficiente', rot = 0, figsize=(12,8))

En las ilustraciones anteriores se aprecia la comparación entre el consumo de leña proyectado para el 2025, con y sin el uso de estufas eficientes

Sin embargo se aprecia que para las veredas El Cajon y Hato Viejo aún se tiene un consumo de leña que no logra se compensando por las estufas de leña

### 4.0 Cambio de Cultivos

El gobierno nacional ha decidido incentivar mediante el pago de un incentivo para cambiar los cultivos de caña y los pastos por cultivos de café que tiene el máximo rendimiento de 
leña. Para esto cuenta con un total de $25.000 millones de pesos; remplazar 1 hectárea de caña vale
$10 millones y 1 hectárea de pastos cuesta $9 millones

Para este fin se pretende realizar el cambio de cultivos en las inmediaciones a los centros poblados

-------------------------------

Actualizando la información de demanda y consumo con la población proyectada

In [20]:
with arcpy.da.UpdateCursor(veredas, ['VEREDA','Consumo','Demanda']) as cursor:
    for row in cursor:
        row[1] = DFVeredasRendimiento.loc[DFVeredasRendimiento['VEREDA'] == row[0]]['Consumo2025'].values[0]
        row[2] = DFVeredasRendimiento.loc[DFVeredasRendimiento['VEREDA'] == row[0]]['Demanda2025'].values[0]
        cursor.updateRow(row)

arcpy.analysis.Select(veredas, veredasDemanda2025, 'Demanda > 0')
arcpy.analysis.Select(usos, usosCambio, "CLASES_USO IN ('Pastos', 'Caña')")




<Result 'C:\\Users\\rfabi\\OneDrive\\Documentos\\Universidad\\00 Esp SIG\\00 LabSIG\\03Taller\\Resultados.gdb\\UsosCambio'>

In [23]:
valorSustitucionCultivos = 0
contador = 1
distancisBuff = 10
while valorSustitucionCultivos < 23000:
    rutaBuffers = os.path.join(GDBResultados,'Buffer_'+str(contador)+'_'+str(distancisBuff))
    rutaTabule = os.path.join(GDBResultados,'Tabule_'+str(contador)+'_'+str(distancisBuff))

    arcpy.analysis.Buffer(pobladosDeficit, rutaBuffers, distancisBuff,"FULL", "ROUND", "ALL")
    arcpy.analysis.TabulateIntersection(usosCambio, 'CLASES_USO', rutaBuffers, rutaTabule, '')
    valorSustitucionCultivos = 0
    with arcpy.da.SearchCursor(rutaTabule, ['CLASES_USO','AREA']) as cursor:
        for row in cursor:
            if 'Pastos' in row[0]:
                valorSustitucionCultivos = (row[1]/10000)*9+valorSustitucionCultivos
            else:
                valorSustitucionCultivos = (row[1]/10000)*10+valorSustitucionCultivos
    print('Valor Actual Inversión en millones '+str(valorSustitucionCultivos))
    contador = contador+1
    distancisBuff = 10 +distancisBuff

bufferFinal = os.path.join(GDBResultados,'Buffer_'+str(contador-1)+'_'+str(distancisBuff-10))
usosActualizados = os.path.join(GDBResultados,'UsosActualizados')
arcpy.analysis.Update(usos, bufferFinal, usosActualizados)

with arcpy.da.UpdateCursor(usosActualizados, ['CLASES_USO']) as cursor:
    for row in cursor:
        if row[0] =='':
            row[0] ='Café'
        cursor.updateRow(row)


Valor Actual Inversión en millones 248.00385136513864
Valor Actual Inversión en millones 312.61405746467676
Valor Actual Inversión en millones 384.10827699758147
Valor Actual Inversión en millones 462.63722797057176
Valor Actual Inversión en millones 548.2346700660235
Valor Actual Inversión en millones 640.9226070760542
Valor Actual Inversión en millones 740.7626305835133
Valor Actual Inversión en millones 847.6237670148698
Valor Actual Inversión en millones 961.5082742086083
Valor Actual Inversión en millones 1082.352405193576
Valor Actual Inversión en millones 1210.1035722371341
Valor Actual Inversión en millones 1344.7848518443857
Valor Actual Inversión en millones 1486.3972244469473
Valor Actual Inversión en millones 1634.9750610354304
Valor Actual Inversión en millones 1790.5707238835446
Valor Actual Inversión en millones 1953.2579994252128
Valor Actual Inversión en millones 2123.076699355855
Valor Actual Inversión en millones 2300.115565455547
Valor Actual Inversión en millones 2

Se realiza la actualización de cambio de uso con la función update y se cálcula nuevamente el rendimiento de los usos con el fin de determinar si se continua manteniendo la demanda de leña

In [24]:
usosVeredaActualizado = os.path.join(GDBResultados,'UsosVeredasActualizado')
arcpy.analysis.TabulateIntersection(veredas, 'VEREDA', usosActualizados, usosVeredaActualizado, 'CLASES_USO')
veredasUSOData = arcpy.da.TableToNumPyArray(usosVeredaActualizado,('VEREDA','CLASES_USO','AREA'))
DFVeredadUSO = pd.DataFrame(veredasUSOData).infer_objects()


DFVeredadUSO = pd.DataFrame(veredasUSOData).infer_objects()
DFRendimientos = pd.DataFrame(tablaRendimientoData).infer_objects()
DFPoblacion = pd.DataFrame(tablaPoblacionData).infer_objects()

DFVeredadUSO['AREA'] = DFVeredadUSO['AREA'].apply(lambda x: round(x/10000,5))
DFVeredasRendimiento = pd.merge(DFVeredadUSO, DFRendimientos, on = 'CLASES_USO', how='left')
DFVeredasRendimiento['Rendimiento'] = DFVeredasRendimiento['AREA']*DFVeredasRendimiento['RENDI']
DFVeredasRendimiento = DFVeredasRendimiento.groupby('VEREDA').sum().reset_index()
DFVeredasRendimiento = DFVeredasRendimiento.drop('RENDI', axis=1)
DFVeredasRendimiento.head(10)

,VEREDA,AREA,Rendimiento
0,EL CAJON,7568.59245,5148.959311
1,EL HATO,7101.11925,8885.496879
2,HATO VIEJO,2541.53438,1171.471593
3,OJO DE AGUA,5609.81089,3361.546156
4,PALMERAS,9534.18300,9975.097960
5,PARAMO BAJO,10302.51711,8485.455803
6,PNN,1805.54568,0.000000
7,RASGATA ALTO,8938.87785,10920.872655
8,SALINAS,8569.16141,4554.708296


In [25]:
tablaPoblacionData = arcpy.da.TableToNumPyArray(tablaPoblacion,('VEREDA','POP_05','POP_01','POP_10','POB_2021'))
DataPoblacion = pd.DataFrame(tablaPoblacionData).infer_objects()
DataPoblacion['POB_2025'] = DataPoblacion['POB_2021']+((DataPoblacion['POB_2021']-DataPoblacion['POP_01'])/(2021-2001))*(2025-2021)
DFVeredasRendimiento = pd.merge(DFVeredasRendimiento, DataPoblacion, on = 'VEREDA', how='left')
DFVeredasRendimiento['Consumo']= DFVeredasRendimiento['POB_2025']*0.5*0.97
DFVeredasRendimiento.head(10)

,VEREDA,AREA,Rendimiento,POP_05,POP_01,POP_10,POB_2021,POB_2025,Consumo
0,EL CAJON,7568.59245,5148.959311,553,162,1493,6563,7843.2,3803.952
1,EL HATO,7101.11925,8885.496879,2127,431,2877,10220,12177.8,5906.233
2,HATO VIEJO,2541.53438,1171.471593,681,389,1222,2621,3067.4,1487.689
3,OJO DE AGUA,5609.81089,3361.546156,696,402,1333,2964,3476.4,1686.054
4,PALMERAS,9534.18300,9975.097960,784,285,1553,4809,5713.8,2771.193
5,PARAMO BAJO,10302.51711,8485.455803,973,678,1617,6124,7213.2,3498.402
6,PNN,1805.54568,0.000000,0,0,0,0,0.0,0.000
7,RASGATA ALTO,8938.87785,10920.872655,610,520,1049,2677,3108.4,1507.574
8,SALINAS,8569.16141,4554.708296,635,346,1273,4300,5090.8,2469.038
